In [ ]:
# imports
import yfinance as yf

import pandas
from pandas_datareader import data
import pandas_datareader as pdr
import pandas_datareader.data as web
import numpy

import datetime
import dateutil
from dateutil.relativedelta import *

from bokeh.plotting import figure , show , output_file
from bokeh.plotting import reset_output
from bokeh.models import HoverTool
from bokeh.models.formatters import TickFormatter, String, List


In [ ]:
# Load and save data by specfied interval and period

# use reset_output() # for output_file in loops
pandas.set_option("display.max_columns", None)

yf.pdr_override()

now = datetime.datetime.now()
nowStr = now.strftime( "%Y-%m-%d" )
fromOrTill = now + relativedelta( days = -6 ) 
fromOrTillStr = fromOrTill.strftime("%Y-%m-%d")

endOrTill = now + relativedelta( days = -1 ) 
endOrTillStr = fromOrTill.strftime("%Y-%m-%d")

timeStr = now.strftime( '%H:%M' )
dayStr = now.strftime('%A')

intervalMin = "1h"

tickers = [ 'ETH-USD']
# eg: 'AAPL', 'SOL1-USD' , 'ETH-USD', 'GOOGL', 

period = '50d'

#Load new data from source
df = web.DataReader( 
        tickers = tickers , 
        data_source = "iex" , 
#         start = fromOrTillStr , 
#         end = endOrTill ,
        interval = intervalMin ,
        period = period
        ).dropna().sort_index()

df.index = df.index.strftime( '%Y-%m-%d %H:%M:%S' )
df.index = pandas.to_datetime(df.index)

#stores the data locally
df.to_pickle("mins_or_hrs_Stocks")

df = pandas.read_pickle("mins_or_hrs_Stocks")

theStart = df.index[0]
theEnd = df.index[-1]

df.columns= df.columns.str.lower()

df.index.name = 'Datetime'

df

In [ ]:
# Change in ticker price in specified period 

def stock_change():
    change = (df['close'][-1] - df['close'][0]) / df['close'][0]
    change_percent = f"{tickers[0]} ({intervalMin}) changed {change*100//0.001*0.001}% over {period}ays"
    return change_percent
    
print(stock_change())

In [ ]:
# To identify black/red and white/green candles

def up_down_P( closeVal , openVal ):
    if closeVal > openVal : value = "up"
    elif closeVal < openVal : value = "down"
    else : value = "same"
    return value 

def interval_stat(df): 
    df["Interval Status"] = [ up_down_P( closeVal , openVal ) 
                              for closeVal , openVal 
                              in zip( df["close"] , df["open"] ) ]
    return df

interval_stat(df)

print()

In [ ]:
# Middle value and Height of candle body

def avg_of_int(df): 
    df["Price AvgOfInt"] = ( df["open"] + df["close"] ) / 2
    return df

def price_change(df): 
    df["Price Change"] = ( df["close"] - df["open"] )
    return df

avg_of_int(df)
price_change(df)

print()

In [ ]:
# Average of Volume over past 10 Days and comparison to current Volume

def avg_of_volume(df, interval): 
    df["AvgOfVolume10Int"] = ( df["volume"].rolling(window=interval).mean().round(0) )
    return df

def up_down_V10( volNow , volAvg10 ):
    if volAvg10 != numpy.nan :
        if volNow > volAvg10 : value = "up"
        elif volNow < volAvg10 : value = "down"
        else : value = ""
        return value 

def volume_status(df):
    df["Volume Status"] = [ up_down_V10( volNow , volAvg10 ) 
                            for volNow , volAvg10 
                            in zip( df["volume"] , df["AvgOfVolume10Int"] ) ]
    return df


interval = 10
avg_of_volume(df, interval)
volume_status(df)

print()

In [ ]:
# Average of Close over past 10 Days and comparison to current Close

def avg_of_close(df, interval): 
    df["AvgOfClose10Int"] = ( df["close"].rolling(window=interval).mean() )
    return df
    
def up_down_c10( cNow , cAvg10 ):
    if cAvg10 != numpy.nan :
        if cNow > cAvg10 : value = "up"
        elif cNow < cAvg10 : value = "down"
        else : value = ""
        return value 

def close_status(df): 
    df["Close Status"] = [ up_down_c10( cNow , cAvg10 ) 
                           for cNow , cAvg10 
                           in zip( df["close"] , df["AvgOfClose10Int"] ) ]
    return df


interval = 10
avg_of_close(df, interval)
close_status(df)

print()

In [ ]:
# Identify bull or bear market

def bb_expectation( cStatus , vStatus ):
    if cStatus == "up" and vStatus == "up" : value = "bullish"
    elif cStatus == "up" and vStatus == "down" : value = "bull trap - weak hands buying"
    elif cStatus == "down" and vStatus == "up" : value = "bearish"
    elif cStatus == "down" and vStatus == "down" : value = "bear trap - weak hands selling"
    else : value = ""
    return value 

def pv_expectation(df):
    df["Price Volume Expectation"] = [ bb_expectation( cStatus , vStatus ) 
                                       for cStatus , vStatus 
                                       in zip( df["Close Status"] , df["Volume Status"] ) ]
    return df
    
pv_expectation(df) 

print()

In [ ]:
# Exponential Moving Average (EMA) in specified interval periods. 
# EMA which gives more weight to more recent values.

theEMAs = [5,10,12,20,26,35,50,100]

def emas( df, theEMAs ):
    
    for x in theEMAs :
        df[f'EMA {x}'] = df["close"].ewm(
            span = x , min_periods=0 , adjust=False , ignore_na=False ).mean()

        index_no = df.columns.get_loc(f'EMA {x}')
        df.iloc[ 0 : (x-1) , [index_no] ] = numpy.nan
        
    return df


emas( df, theEMAs )

print()

In [ ]:
#Buy or sell by close price comparison to each of EMAs

def emaBuyOrSell( theClose , theEMA ):
    if theEMA != numpy.nan :
        if theClose > theEMA : value = "buy"
        elif theClose < theEMA : value = "sell"
        else : value = ""
        return value 

def emas_BuyOrSell(df):
    for x in theEMAs :
        df[f"EMA{x} Buy or Sell" ] = [ emaBuyOrSell( theClose , theEMA ) 
                                       for theClose , theEMA 
                                       in zip( df["close"] , df[f"EMA {x}"] ) ]
    return df
 
    
emas_BuyOrSell(df)

print()

In [ ]:
#Backtest: Exponential Moving Average (EMA) Crossover Profit/Loss, buy when cross of EMAs, or EMA and close
#Conclusion: Poor buy or sell signals if not combined with other indicators. 

finaltheBalance = []
finaltheInvested = []
finaltheShareMultiplier = []
theCloselist = []
finaltheDates = []

class EMAcrossover:
    def __init__(self, aVal , bVal, cVal ):  
        self.tradeAccountBalance = [] #theBalance
        if len(self.tradeAccountBalance) == 0 : 
            self.tradeAccountBalance.append(aVal)
        self.aVal = aVal
        
        self.tradeAccountBalanceInitial = self.tradeAccountBalance[0]
        
        self.tradeAccountInvested = [] #theInvested
        if len(self.tradeAccountInvested) == 0 : 
            self.tradeAccountInvested.append(bVal)
        self.bVal = bVal
        
        self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

        self.shareCountMultiplier = [] #theMultipler
        if len(self.shareCountMultiplier) == 0 : 
            self.shareCountMultiplier.append(cVal)
        self.cVal = cVal

        self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
        
        self.theDates = [] #theDates
        self.theDates.append(f"{df.index[0]} START")


    def calcProfit(self, ) :
        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        oBal = self.tradeAccountBalanceInitial
        oInv = self.tradeAccountInvestedInitial
        
        value = ( ( ((inv+bal)-(oBal+oInv)) / (oBal+oInv) ) ) * 100
        return value 
        
    def x_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex > 0 :
            emaXtemp = df[f'EMA {x}'][df.index[tIndex-1]]
            emaX1temp = df[f'EMA {x1}'][df.index[tIndex-1]]
            if ( (emaXtemp < emaX1temp) 
                and (emaXtemp != numpy.nan) and (emaX1temp != numpy.nan) ) : return True
            else : return False
            
        else : return False
        
    def x1_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex > 0 :
            emaXtemp = df[f'EMA {x}'][df.index[tIndex-1]]
            emaX1temp = df[f'EMA {x1}'][df.index[tIndex-1]]
            if ( (emaXtemp > emaX1temp) 
                and (emaXtemp != numpy.nan) and (emaX1temp != numpy.nan) ) : return True
            else : return False
        
        else : return False
        
    def emaProfit(self, emaX , emaX1 , theClose , x , x1 , tIndex  ) :
        global finaltheBalance , finaltheInvested, finaltheShareMultiplier 

        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        mult = self.shareCountMultiplier[-1]
        
        #UPDATE INVESTMENT to current market value
        if mult >= 1 and inv > 0 and (theClose != numpy.nan) :                      
            self.tradeAccountInvested[-1] = theClose*mult
        inv = self.tradeAccountInvested[-1]

        x_lessYest = self.x_isLessYesterday(x , x1 , tIndex)
        x1_lessYest = self.x1_isLessYesterday(x , x1 , tIndex)

        # Close and EMA crossover
#         if tIndex > 0 :
#             emaX1Yest = df[f'EMA {x1}'][df.index[tIndex-1]]
#             theCloseYest = df['close'][df.index[tIndex-1]]
            
            #SELL if market price turns less than X1 (OPTIONAL)
#             if ( (theClose < emaX1) and (theCloseYest > emaX1Yest) and (inv > 0) 
                
#                 and (emaX1 != numpy.nan) and (theCloseYest != numpy.nan) 
#                 and (emaX1Yest != numpy.nan) and (theClose != numpy.nan) ) :
                
#                 mult = int( inv / theClose )     
#                 bal = bal + inv - (theClose*mult*0.003) #fee
#                 inv = 0.00  
#                 mult = 0.00

#                 self.tradeAccountBalance.append(bal)
#                 self.tradeAccountInvested.append(inv)
#                 self.shareCountMultiplier.append(mult)

#                 tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
#                 self.theDates.append(f"{tDate} SELL")

            #Buy if market price turns greater than X1 (OPTIONAL)
#             elif ( (theClose > emaX1) and (theCloseYest < emaX1Yest) and (bal >= theClose) and (emaX > emaX1)
                  
#                      and (theClose != numpy.nan) and (emaX1 != numpy.nan) and (emaX != numpy.nan) 
#                      and (theCloseYest != numpy.nan) and (emaX1Yest != numpy.nan)  
#                  ) :

#                 #Better buy price
# #                 if tIndex > 0 :
# #                     theClose = df["close"][df.index[tIndex-1]]

#                 mult = int( bal / theClose )     
#                 bal = bal - theClose*mult - (theClose*mult*0.003)    #buy as many shares as balance available minus fee
#                 inv = inv + theClose*mult         

#                 self.tradeAccountBalance.append(bal)
#                 self.tradeAccountInvested.append(inv)
#                 self.shareCountMultiplier.append(mult)

#                 tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
#                 self.theDates.append(f"{tDate} BUY")

#             else:
#                 pass

        #BUY if EMA x turns greater than EMA x1 at market price
        if ( (emaX > emaX1) and x_lessYest and (bal >= theClose)
                and (emaX != numpy.nan) and (emaX1 != numpy.nan) and (theClose!=numpy.nan) 
           ) :

            #Better buy price
#             if tIndex > 0 : theClose = df["close"][df.index[tIndex-1]]

            mult = int( bal / theClose )     
            bal = bal - theClose*mult - (theClose*mult*0.003)    #buy as many shares as balance available minus fee
            inv = inv + theClose*mult         

            if self.shareCountMultiplier[-1] > 0 :
                mult = mult + self.shareCountMultiplier[-1]

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
            self.theDates.append(f"{tDate} BUY")

        #SELL if EMA x turns lesser than EMA x1 at market price
        elif ( (emaX < emaX1) and x1_lessYest and (inv > 0) 
                   and (emaX != numpy.nan) and (emaX1!= numpy.nan)
              
#                   #Stoploss condition
#                 or ((inv > 0) 
#                    and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*0.9) >= (inv+bal)) 
#                    and (self.tradeAccountInvested[-2] != numpy.nan)   )
#                   #Takeprofit condition
#                 or ((inv > 0) 
#                    and ((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*1.15 < (inv+bal)) 
#                    and (self.tradeAccountInvested[-2] != numpy.nan)   )    
              
              )  :
            mult = int( inv / theClose )     
            bal = bal + inv - (theClose*mult*0.003) #fee
            inv = 0.00  
            mult = 0

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
            self.theDates.append(f"{tDate} SELL")

        else:
            pass

        #PROFIT calculator
        value = self.calcProfit()

        #RESET for next loop for different crossover
        if tIndex >= len(df) - 1 : 
            #self.indexPrint( tIndex )
            finaltheBalance.append(self.tradeAccountBalance[0:-1])
            self.tradeAccountBalance.clear()
            self.tradeAccountBalance.append(self.aVal)

            self.theBalanceInitial = self.tradeAccountBalance[0]

            finaltheInvested.append(self.tradeAccountInvested[0:-1])
            self.tradeAccountInvested.clear()
            self.tradeAccountInvested.append(self.bVal)

            self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

            finaltheShareMultiplier.append(self.shareCountMultiplier[0:-1])
            self.shareCountMultiplier.clear()
            self.shareCountMultiplier.append(self.cVal)

            self.shareCountMultiplierInitial = self.shareCountMultiplier[0]

            finaltheDates.append(self.theDates[0:-1])
            self.theDates.clear()
            self.theDates.append(f"{df.index[0]} START")

        return value 

    def indexPrint(self, tIndex ) :
        print(tIndex)

    def emaProfitDFmake(self, ): 
        global finaltheBalance , finaltheInvested
        theEMAsP = [5,10,20,50,12,5]
        theEMAs2P = [10,20,50,100,26,35]

        for x , x1 in zip(theEMAsP,theEMAs2P) :
            df[f'EMA {x}-{x1} crossover Profit %'] =  [ 
                self.emaProfit( emaX , emaX1 , theClose , x , x1, tIndex )
                for tIndex , (emaX , emaX1 , theClose)
                in enumerate( zip( df[f'EMA {x}'] , df[f'EMA {x1}'] , df["close"] ) ) ]
                
            print(f'EMA {x}-{x1} crossover Profit % {tickers[0]}')
#             print(f"Date : {finaltheDates[-1]}")
#             print(f"Balance : {finaltheBalance[-1]}")
#             print(f"Invested : {finaltheInvested[-1]}")
#             print(f"Multiplier : {finaltheShareMultiplier[-1]}")
            print(f"Profit : {df[f'EMA {x}-{x1} crossover Profit %'][df.index[-1]]}")
            print("")
            print("")
              

print(stock_change())
print()

starting_balance = 100000.00   

crossProfit = EMAcrossover(starting_balance , 0.00, 0)

crossProfit.emaProfitDFmake()




In [ ]:
# Comparison of middle of a candle body with the candle before it. 

def up_down_P_Avg( tIndex , avgPrice ):
    if tIndex > 0 :
        avgPriceYest = df["Price AvgOfInt"][df.index[tIndex-1]]
        if avgPrice > avgPriceYest : value = "up"
        elif avgPrice < avgPriceYest : value = "down"
        else : value = ""
        return value 
    
def avg_price_status(df):    
    df["AVG Price Status"] = [ up_down_P_Avg( tIndex , avgPrice ) 
                               for tIndex , avgPrice 
                               in enumerate( df["Price AvgOfInt"] )  ]
    #                            in enumerate( df["close"] )  ]
    return df


avg_price_status(df)

print()

In [ ]:
# Change in trend with confirmation.

def isBearToBullAvgP( tIndex ) :
    if tIndex >= 5 :
        try: 
            avgPStatus = df["AVG Price Status"]
            if (( avgPStatus[df.index[tIndex]]== "up") and
                ( avgPStatus[df.index[tIndex-1]]== "up") and 
                ( avgPStatus[df.index[tIndex-2]]== "down") and 
                ( avgPStatus[df.index[tIndex-3]]== "down") and 
                ( avgPStatus[df.index[tIndex-4]]== "down") and 
                ( avgPStatus[df.index[tIndex-5]]== "down") and 
                ( avgPStatus[df.index[tIndex-6]]== "down") ) : return True
        except: pass  
    else: return False

def isBullToBearAvgP( tIndex ) :
    if tIndex >= 5 :
        try: 
            avgPStatus = df["AVG Price Status"]
            if (( avgPStatus[df.index[tIndex]]== "down") and
                ( avgPStatus[df.index[tIndex-1]]== "down") and 
                ( avgPStatus[df.index[tIndex-2]]== "up") and 
                ( avgPStatus[df.index[tIndex-3]]== "up") and 
                ( avgPStatus[df.index[tIndex-4]]== "up") and 
                ( avgPStatus[df.index[tIndex-5]]== "up") and 
                ( avgPStatus[df.index[tIndex-6]]== "up") ) : return True
        except: pass  
    else: return False

def is_BtoB_or_BtoB_AvgP( tIndex, avgPStatus ) :
    try: 
        if isBearToBullAvgP( tIndex ) == True : return "Bear to Bull"
        elif isBullToBearAvgP( tIndex ) == True : return "Bull to Bear"
        else : pass
    except: pass

def bb_avgp_status(df):
    df["BtoB AvgP Status"] = [ 
            is_BtoB_or_BtoB_AvgP( tIndex , avgPStatus ) 
            for tIndex , avgPStatus in enumerate( df["AVG Price Status"] ) ]
    return df


bb_avgp_status(df)

print()

In [ ]:
df[ "BtoB AvgP Status"].describe()


In [ ]:
# Backtest: Bull to Bear trend Profit/Loss
# Conclusion: Doesn't take into account the larger picture of stock with other indicators.

finaltheBalance = []
finaltheInvested = []
finaltheShareMultiplier = []
finaltheDates = []

class AvgPProfit:
    def __init__(self, aVal , bVal, cVal ):  
        self.tradeAccountBalance = [] #theBalance
        if len(self.tradeAccountBalance) == 0 : 
            self.tradeAccountBalance.append(aVal)
        self.aVal = aVal
        
        self.tradeAccountBalanceInitial = self.tradeAccountBalance[0]
        
        self.tradeAccountInvested = [] #theInvested
        if len(self.tradeAccountInvested) == 0 : 
            self.tradeAccountInvested.append(bVal)
        self.bVal = bVal
        
        self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

        self.shareCountMultiplier = [] #theMultiplier
        if len(self.shareCountMultiplier) == 0 : 
            self.shareCountMultiplier.append(cVal)
        self.cVal = cVal
        
        self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
        
        self.theDates = [] #theDates
        self.theDates.append(f"{theStart} START")

    def calcProfit(self, ) :
        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        oBal = self.tradeAccountBalanceInitial
        oInv = self.tradeAccountInvestedInitial
    
        value = ( ((inv+bal) - (oBal+oInv)) / (oBal+oInv) ) * 100
        return value 
        
    def avgpProfit(self, btobS , avgP , theClose , tIndex  ) :
        global finaltheBalance , finaltheInvested, finaltheShareMultiplier, finaltheDates

        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        mult = self.shareCountMultiplier[-1]
        
        #UPDATE INVESTMENT to current market value
        if (mult >= 1) and (inv > 0) and (theClose != numpy.nan) :                      
            self.tradeAccountInvested[-1] = theClose*mult
        inv = self.tradeAccountInvested[-1]

        #BUY if Bear to Bull
        if ( (btobS == "Bear to Bull") and (bal >= theClose) and (theClose != numpy.nan) ) :
            
            #Better buy price
#             if tIndex > 0 : theClose = df["close"][df.index[tIndex-1]]

            mult = int( bal / theClose )     
            bal = bal - theClose*mult - (theClose*mult*0.003)     #buy as many shares as balance available minus fee
            inv = inv + theClose*mult         
                
            if self.shareCountMultiplier[-1] > 0 :
                mult = mult + self.shareCountMultiplier[-1]
            
            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)
            
            tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
            self.theDates.append(f"{tDate} BUY")
            
        #SELL if Bull to Bear
        elif ( ((btobS == "Bull to Bear") and (inv > 0))
              
                  #Stoploss condition
                or ((inv > 0) 
                   and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*0.95) >= (inv+bal)) 
                   and (self.tradeAccountInvested[-2] != numpy.nan)   )
                  #Takeprofit condition
                or ((inv > 0) 
                   and ((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*1.02 < (inv+bal)) 
                   and (self.tradeAccountInvested[-2] != numpy.nan)   )    
              
             )  :
            mult = int( inv / theClose )     
            bal = bal + inv - (theClose*mult*0.003)
            inv = 0.00  
            mult = 0

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)
            
            tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
            self.theDates.append(f"{tDate} SELL")
        else:
            pass
               
        #PROFIT calculator
        value = self.calcProfit()
        
        #RESET for next loop for different crossover
        if tIndex >= len(df) - 1 : 
            #self.indexPrint( tIndex )
            finaltheBalance.append(self.tradeAccountBalance[0:-1])
            self.tradeAccountBalance.clear()
            self.tradeAccountBalance.append(self.aVal)
            
            self.theBalanceInitial = self.tradeAccountBalance[0]
            
            finaltheInvested.append(self.tradeAccountInvested[0:-1])
            self.tradeAccountInvested.clear()
            self.tradeAccountInvested.append(self.bVal)
            
            self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

            finaltheShareMultiplier.append(self.shareCountMultiplier[0:-1])
            self.shareCountMultiplier.clear()
            self.shareCountMultiplier.append(self.cVal)
            
            self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
            
            finaltheDates.append(self.theDates[0:-1])
            self.theDates.clear()
            self.theDates.append(f"{theStart} START")

        return value 
    
    def indexPrint(self, tIndex ) :
        print(tIndex)

    def avgpProfitDFmake(self, ): 
        global finaltheBalance , finaltheInvested , finaltheShareMultiplier

        df['Avg P Profit %'] =  [ 
                self.avgpProfit( btobS , avgP , theClose , tIndex  )
                for tIndex , (btobS , avgP , theClose)
                in enumerate( zip( df["BtoB AvgP Status"] , df["Price AvgOfInt"] , df["close"] ) )   ]

        print(f'Avg P Profit % {tickers[0]}')
#         print(f"Date : {finaltheDates[-1]}")
#         print(f"Balance : {finaltheBalance[-1]}")
#         print(f"Invested : {finaltheInvested[-1]}")
#         print(f"Multiplier : {finaltheShareMultiplier[-1]}")
        print(f"Profit : {df['Avg P Profit %'][df.index[-1]]}")
        print("")
        print("")

print(stock_change())
print()

starting_balance = 100000.00   

avgpProfit = AvgPProfit(starting_balance , 0.00, 0)

avgpProfit.avgpProfitDFmake()



In [ ]:
# FOR .CSV add below the above cell or other Backtests

# flat_listB = [item for sublist in finaltheBalance for item in sublist]
# flat_listI = [item for sublist in finaltheInvested for item in sublist]
# flat_listM = [item for sublist in finaltheShareMultiplier for item in sublist]
# dft = pandas.DataFrame(
#     list(zip(flat_listB, flat_listI, flat_listM, finalProfit)) , 
#     columns=['New Balance', 'New Invested', 'multiplier', 'profit'])
# dft.to_csv("to check.csv")

In [ ]:
# Relative Strength Index (RSI) to indicate oversold or overbought ticker by specified period. 
# 14 interval is standard.

rsi_period = 14

def rma(x, rsi_period, y0):
    n = rsi_period
    a = (n-1) / n
    ak = a**numpy.arange(len(x)-1, -1, -1)
    return numpy.r_[numpy.full(n, numpy.nan), y0, numpy.cumsum(ak * x) / ak / n + y0 * a**numpy.arange(1, len(x)+1)]

def close_change(df):
    df['Close Change'] = df['close'].diff()
    return df

def points_gained(df):
    close_change(df)
    df["Points Gained"] = df['Close Change'].mask(df['Close Change'] < 0, 0.0)
    return df

def points_lost(df):
    close_change(df)
    df["Points Lost"] = abs(df['Close Change'].mask(df['Close Change'] > 0, -0.0))
    return df

def avg_points_gained(df):
    points_gained(df)
    df["AVG Points Gained"] = rma(
            df["Points Gained"][rsi_period+1:].to_numpy(), 
            rsi_period, 
            numpy.nansum(df["Points Gained"].to_numpy()[:rsi_period+1])  / rsi_period )
    return df

def avg_points_lost(df):
    points_lost(df)
    df["AVG Points Lost"] = rma(
            df["Points Lost"][rsi_period+1:].to_numpy(), 
            rsi_period, 
            numpy.nansum(df["Points Lost"].to_numpy()[:rsi_period+1])  / rsi_period )
    return df

def rsi(df):
    avg_points_gained(df)
    avg_points_lost(df)
    df["RS"] = df["AVG Points Gained"] / df["AVG Points Lost"] 
    df["RSI"] = 100 - ( 100 / ( 1 + df["RS"] ) ) 
    return df


rsi(df)

print()

In [ ]:
# Marking if a ticker is oversold or overbought by specified threshold below.

def isOversoldRSI( tIndex , rsi_osval ) :
    try: 
        rsiVal = df["RSI"]
        if ( rsiVal[df.index[tIndex]] <= rsi_osval ) : return True
    except: pass 
    
def isOverboughtRSI( tIndex , rsi_obval ) :
    try: 
        rsiVal = df["RSI"]
        if ( rsiVal[df.index[tIndex]] >= rsi_obval ) : return True
    except: pass  

def is_Oversold_or_Overbought_RSI( tIndex , rsiVal, rsi_osval, rsi_obval ) :
    try: 
        if isOversoldRSI( tIndex , rsi_osval ) == True : return "oversold"
        elif isOverboughtRSI( tIndex , rsi_obval ) == True : return "overbought"
        else : pass
    except: pass

def rsi_status(df, rsi_osval, rsi_obval):    
    df[f"RSI Status {rsi_osval}-{rsi_obval}"] = [ 
        
            is_Oversold_or_Overbought_RSI( tIndex , rsiVal, rsi_osval, rsi_obval ) 
            for tIndex , rsiVal 
            in enumerate( df["RSI"] )   
    ]
    return df
    
    
rsi_osval = 25
rsi_obval = 75

rsi_status(df, rsi_osval, rsi_obval)

df.index[df[f"RSI Status {rsi_osval}-{rsi_obval}"] == "oversold"].tolist()
# If RSI remains below 20 for extended period then short in that direction(bearish)
# If RSI remains above 80 for extended period then buy in that direction(bullish)

print()

In [ ]:
# MACD (Moving Average Convergence Divergence)
# We find the difference between two EMA periods and find the EMA of that difference. 
# Here I'm taking EMA 9 and 5 of the EMA Diff

def ema_diff(df, theEMAsP, theEMAs2P):
    for x , x1 in zip(theEMAsP,theEMAs2P) :
        df[f'{x}-{x1} EMA diff'] = df[f'EMA {x}'].sub(df[f'EMA {x1}'], axis = 0) 
    return df


def ema_diff_ema9(df, theEMAsP, theEMAs2P):
    ema_diff(df, theEMAsP, theEMAs2P)
    
    for x , x1 in zip(theEMAsP,theEMAs2P) :
        df[f'{x}-{x1} EMA diff EMA 9'] = df[f'{x}-{x1} EMA diff'].ewm(
            span = 9 , min_periods=0 , adjust=False , ignore_na=False ).mean()

        index_no = df.columns.get_loc(f'{x}-{x1} EMA diff EMA 9')
        df.iloc[ 0 : 8 , [index_no] ] = numpy.nan
        
    return df

def ema_diff_ema5(df, theEMAsP, theEMAs2P):
    ema_diff(df, theEMAsP, theEMAs2P)
    for x , x1 in zip(theEMAsP,theEMAs2P) :
        df[f'{x}-{x1} EMA diff EMA 5'] = df[f'{x}-{x1} EMA diff'].ewm(
            span = 5 , min_periods=0 , adjust=False , ignore_na=False ).mean()

        index_no = df.columns.get_loc(f'{x}-{x1} EMA diff EMA 5')
        df.iloc[ 0 : 4 , [index_no] ] = numpy.nan
    return df

def emaMACDBuyOrSell( theXX1DiffEMA9 , theXX1Diff ):
    if theXX1DiffEMA9 != numpy.nan :
        if theXX1Diff > theXX1DiffEMA9 : value = "buy"
        elif theXX1Diff < theXX1DiffEMA9 : value = "sell"
        else : value = ""
        return value 
    
def macd9_buy_sell(df, theEMAsP, theEMAs2P):
    ema_diff_ema9(df, theEMAsP, theEMAs2P)
    
    for x , x1 in zip(theEMAsP,theEMAs2P) :
        df[f"MACD9 {x}-{x1} Buy or Sell"] = [ 
                emaMACDBuyOrSell( theXX1DiffEMA9 , theXX1Diff )
                for theXX1DiffEMA9 , theXX1Diff 
                in zip( df[f'{x}-{x1} EMA diff EMA 9'] , 
                        df[f'{x}-{x1} EMA diff']   )   ]
    return df

def macd5_buy_sell(df, theEMAsP, theEMAs2P):
    ema_diff_ema5(df, theEMAsP, theEMAs2P)
    
    for x , x1 in zip(theEMAsP,theEMAs2P) :
        df[f"MACD5 {x}-{x1} Buy or Sell"] = [ 
                emaMACDBuyOrSell( theXX1DiffEMA5 , theXX1Diff )
                for theXX1DiffEMA5 , theXX1Diff 
                in zip( df[f'{x}-{x1} EMA diff EMA 5'] , 
                        df[f'{x}-{x1} EMA diff']   )   ]
    return df

theEMAsP = [12,5,5,10,20,50]
theEMAs2P = [26,35,10,20,50,100]

macd9_buy_sell(df, theEMAsP, theEMAs2P)
macd5_buy_sell(df, theEMAsP, theEMAs2P)

print()

In [ ]:
# Backtest: MACD9 Profit/Loss, buying and selling at crossover of the EMA Diff and its EMA. 
# Conclusion: Buy and Sell signal is not specific enough. Too many trades means too much profit lost to fees. 

finaltheBalance = []
finaltheInvested = []
finaltheShareMultiplier = []
finaltheDates = []


class MACD9Profit:
    def __init__(self, aVal , bVal, cVal ):  
        self.tradeAccountBalance = [] #theBalance
        if len(self.tradeAccountBalance) == 0 : 
            self.tradeAccountBalance.append(aVal)
        self.aVal = aVal
        
        self.tradeAccountBalanceInitial = self.tradeAccountBalance[0]
        
        self.tradeAccountInvested = [] #theInvested
        if len(self.tradeAccountInvested) == 0 : 
            self.tradeAccountInvested.append(bVal)
        self.bVal = bVal
        
        self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

        self.shareCountMultiplier = [] #theMultiplier
        if len(self.shareCountMultiplier) == 0 : 
            self.shareCountMultiplier.append(cVal)
        self.cVal = cVal
        
        self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
        
        self.theDates = [] #theDates
        self.theDates.append(f"{theStart} START")

    def calcProfit(self, ) :
        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        oBal = self.tradeAccountBalanceInitial
        oInv = self.tradeAccountInvestedInitial
    
        value = ( (((inv+bal) - (oBal+oInv)) / (oBal+oInv) ) ) * 100
        return value 
        
    def theXX1Diff_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex != 0:
            theXX1Diff_temp = df[f'{x}-{x1} EMA diff'][df.index[tIndex-1]]
            theXX1DiffEMA9_temp = df[f'{x}-{x1} EMA diff EMA 9'][df.index[tIndex-1]]
            if (theXX1DiffEMA9_temp != numpy.nan) and (theXX1Diff_temp < theXX1DiffEMA9_temp) :
                return True
            else: return False 
        else: return False 
                
    def theXX1DiffEMA9_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex != 0:
            theXX1Diff_temp = df[f'{x}-{x1} EMA diff'][df.index[tIndex-1]]
            theXX1DiffEMA9_temp = df[f'{x}-{x1} EMA diff EMA 9'][df.index[tIndex-1]]
            if (theXX1DiffEMA9_temp != numpy.nan) and (theXX1Diff_temp > theXX1DiffEMA9_temp) :
                return True
            else: return False 
        else: return False
        
    def macd9Profit(self, theXX1DiffEMA9 , theXX1Diff , theClose , x , x1 , tIndex  ) :
        global finaltheBalance , finaltheInvested, finaltheShareMultiplier 

        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        mult = self.shareCountMultiplier[-1]
        
        #UPDATE INVESTMENT to current market value
        if mult >= 1 and inv > 0 : self.tradeAccountInvested[-1] = theClose*mult
        inv = self.tradeAccountInvested[-1]

        theXX1Diff_lessYest = self.theXX1Diff_isLessYesterday(x , x1 , tIndex)
        theXX1DiffEMA9_lessYest = self.theXX1DiffEMA9_isLessYesterday(x , x1 , tIndex)

        #BUY if EMA x turns greater than x1 market price
        if ( (theXX1Diff > theXX1DiffEMA9 ) and ( theXX1Diff_lessYest == True) 
                 and (bal >= theClose) and (theXX1DiffEMA9 != numpy.nan) 
           
           ) :

            #Better buy price when tIndex-1 that is previous close
#             if tIndex > 0 : theClose = df["close"][df.index[tIndex-1]]

            mult = int( bal / theClose )     
            bal = bal - theClose*mult - (theClose*mult*0.003)    #buy as many shares as balance available minus fee
            inv = inv + theClose*mult

            if self.shareCountMultiplier[-1] > 0 :
                mult = mult + self.shareCountMultiplier[-1]

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} BUY")


        # SELL if EMA x turns lesser than x1 market price
        # or SELL if down 1%
        # or SELL if made atleast 2% profit
        elif ( ((theXX1Diff < theXX1DiffEMA9) and ( theXX1DiffEMA9_lessYest == True) 
                    and (inv > 0) and (theXX1DiffEMA9 != numpy.nan)) 
              
                #Stoploss
                or ( (inv > 0) 
                    and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*0.99) >= (inv+bal)) 
                    and (self.tradeAccountInvested[-2] != numpy.nan) )
                #Takeprofit
                or ( (inv > 0) 
                    and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*1.015) < (inv+bal)) 
                    and (self.tradeAccountInvested[-2] != numpy.nan) )
              
             )  :
            mult = int( inv / theClose )     
            bal = bal + inv - (theClose*mult*0.003) #0.3% fees
            inv = 0.00  
            mult = 0

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} SELL")
            
            
        else:
            pass

        #PROFIT calculator
        value = self.calcProfit()

        #RESET for next loop for different crossover
        if tIndex >= len(df) - 1 : 
            #self.indexPrint( tIndex )
            finaltheBalance.append(self.tradeAccountBalance[0:-1])
            self.tradeAccountBalance.clear()
            self.tradeAccountBalance.append(self.aVal)

            self.theBalanceInitial = self.tradeAccountBalance[0]

            finaltheInvested.append(self.tradeAccountInvested[0:-1])
            self.tradeAccountInvested.clear()
            self.tradeAccountInvested.append(self.bVal)

            self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

            finaltheShareMultiplier.append(self.shareCountMultiplier[0:-1])
            self.shareCountMultiplier.clear()
            self.shareCountMultiplier.append(self.cVal)

            self.shareCountMultiplierInitial = self.shareCountMultiplier[0]

            finaltheDates.append(self.theDates[0:-1])
            self.theDates.clear()
            self.theDates.append(f"{theStart} START")

        return value 
    
    def indexPrint(self, tIndex ) :
        print(tIndex)

    def macd9ProfitDFmake(self, ): 
        global finaltheBalance , finaltheInvested
        theEMAsP = [12,5,5,10,20,50]
        theEMAs2P = [26,10,35,20,50,100]

        for x , x1 in zip(theEMAsP,theEMAs2P) :
            df[f'{x}-{x1} Diff EMA9 crossover Profit %'] =  [ 
                    self.macd9Profit( theXX1DiffEMA9 , theXX1Diff , theClose , x , x1 , tIndex  )
                       for tIndex , (theXX1DiffEMA9 , theXX1Diff, theClose)
                       in enumerate( 
                           zip( df[f'{x}-{x1} EMA diff EMA 9'] , 
                                df[f'{x}-{x1} EMA diff'] ,
                                df["close"]   )   )   ]

            print(f'{x}-{x1} Diff EMA9 crossover Profit % {tickers[0]}')
#             print(f"Date : {finaltheDates[-1]}")
#             print(f"Balance : {finaltheBalance[-1]}")
#             print(f"Invested : {finaltheInvested[-1]}")
#             print(f"Multiplier : {finaltheShareMultiplier[-1]}")
            print(f"Profit : {df[f'{x}-{x1} Diff EMA9 crossover Profit %'][df.index[-1]]}")
            print("")
            print("")
                
print(stock_change())
print()


starting_balance = 10000.00

macd9Profit = MACD9Profit(starting_balance , 0, 0)

macd9Profit.macd9ProfitDFmake()



In [ ]:
# Backtest: MACD5 Profit/Loss, buying and selling at crossover of the EMA Diff and its EMA. 
# Conclusion: Buy and Sell signal is not specific enough. Too many trades means too much profit lost to fees. 

finaltheBalance = []
finaltheInvested = []
finaltheShareMultiplier = []
finaltheDates = []


class MACD5Profit:
    def __init__(self, aVal , bVal, cVal ):  
        self.tradeAccountBalance = []
        if len(self.tradeAccountBalance) == 0 : 
            self.tradeAccountBalance.append(aVal)
        self.aVal = aVal
        
        self.tradeAccountBalanceInitial = self.tradeAccountBalance[0]
        
        self.tradeAccountInvested = [] #theInvested
        if len(self.tradeAccountInvested) == 0 : 
            self.tradeAccountInvested.append(bVal)
        self.bVal = bVal
        
        self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

        self.shareCountMultiplier = []
        if len(self.shareCountMultiplier) == 0 : 
            self.shareCountMultiplier.append(cVal)
        self.cVal = cVal
        
        self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
        
        self.theDates = []
        self.theDates.append(f"{theStart} START")

    def calcProfit(self, ) :
        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        oBal = self.tradeAccountBalanceInitial
        oInv = self.tradeAccountInvestedInitial
    
        value = ( (((inv+bal) - (oBal+oInv)) / (oBal+oInv) ) ) * 100
        return value 
        
    def theXX1Diff_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex != 0:
            theXX1Diff_temp = df[f'{x}-{x1} EMA diff'][df.index[tIndex-1]]
            theXX1DiffEMA5_temp = df[f'{x}-{x1} EMA diff EMA 5'][df.index[tIndex-1]]
            if (theXX1DiffEMA5_temp != numpy.nan) and (theXX1Diff_temp < theXX1DiffEMA5_temp) :
                return True
            else: return False 
        else: return False 
                
    def theXX1DiffEMA5_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex != 0:
            theXX1Diff_temp = df[ f'{x}-{x1} EMA diff'][df.index[tIndex-1]]
            theXX1DiffEMA5_temp = df[f'{x}-{x1} EMA diff EMA 5'][df.index[tIndex-1]]
            if (theXX1DiffEMA5_temp != numpy.nan) and (theXX1Diff_temp > theXX1DiffEMA5_temp) :
                return True
            else: return False 
        else: return False
        
    def macd5Profit(self, theXX1DiffEMA5 , theXX1Diff , theClose , x , x1 , tIndex  ) :
        global finaltheBalance , finaltheInvested, finaltheShareMultiplier, finaltheDates 

        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        mult = self.shareCountMultiplier[-1]
        
        #UPDATE INVESTMENT to current market value
        if mult >= 1 and inv > 0 : self.tradeAccountInvested[-1] = theClose*mult
        inv = self.tradeAccountInvested[-1]

        theXX1Diff_lessYest = self.theXX1Diff_isLessYesterday(x , x1 , tIndex)
        theXX1DiffEMA5_lessYest = self.theXX1DiffEMA5_isLessYesterday(x , x1 , tIndex)

        #BUY if EMA x turns greater than x1 market price
        if ( (theXX1Diff > theXX1DiffEMA5 ) and ( theXX1Diff_lessYest == True) 
             and (bal >= theClose) and (theXX1DiffEMA5 != numpy.nan)) :

            #Better buy price at tIndex-1 which is previous close
#             if tIndex > 0 : theClose = df["close"][df.index[tIndex-1]]

            mult = int( bal / theClose )     
            bal = bal - theClose*mult - (theClose*mult*0.003)     #buy as many shares as balance available
            inv = inv + theClose*mult

            if self.shareCountMultiplier[-1] > 0 :
                mult = mult + self.shareCountMultiplier[-1]

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
            self.theDates.append(f"{tDate} BUY")


        #SELL if EMA x turns lesser than x1 market price
        elif ( ((theXX1Diff < theXX1DiffEMA5) and ( theXX1DiffEMA5_lessYest == True) 
               and (inv > 0) and (theXX1DiffEMA5 != numpy.nan)) 
              
                #Stoploss
                or ( (inv > 0) 
                    and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*0.99) >= (inv+bal)) 
                    and (self.tradeAccountInvested[-2] != numpy.nan) )
                #Takeprofit
                or ( (inv > 0) 
                    and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*1.01) < (inv+bal)) 
                    and (self.tradeAccountInvested[-2] != numpy.nan) )
              
             )  :
            mult = int( inv / theClose )     
            bal = bal + inv - (theClose*mult*0.003)
            inv = 0.00  
            mult = 0

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d' )
            self.theDates.append(f"{tDate} SELL")

        else: pass

        #PROFIT calculator
        value = self.calcProfit()

        #RESET for next loop for different crossover
        if tIndex >= len(df) - 1 : 
            #self.indexPrint( tIndex )
            finaltheBalance.append(self.tradeAccountBalance[0:-1])
            self.tradeAccountBalance.clear()
            self.tradeAccountBalance.append(self.aVal)

            self.theBalanceInitial = self.tradeAccountBalance[0]

            finaltheInvested.append(self.tradeAccountInvested[0:-1])
            self.tradeAccountInvested.clear()
            self.tradeAccountInvested.append(self.bVal)

            self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

            finaltheShareMultiplier.append(self.shareCountMultiplier[0:-1])
            self.shareCountMultiplier.clear()
            self.shareCountMultiplier.append(self.cVal)

            self.shareCountMultiplierInitial = self.shareCountMultiplier[0]

            finaltheDates.append(self.theDates[0:-1])
            self.theDates.clear()
            self.theDates.append(f"{theStart} START")

        return value 
    
    def indexPrint(self, tIndex ) :
        print(tIndex)

    def macd5ProfitDFmake(self, ): 
        global finaltheBalance , finaltheInvested
        theEMAsP = [12,5,5,10,20,50]
        theEMAs2P = [26,35,10,20,50,100]

        for x , x1 in zip(theEMAsP,theEMAs2P) :
            df[f'{x}-{x1} Diff EMA5 crossover Profit %'] =  [ 
                    self.macd5Profit( theXX1DiffEMA5 , theXX1Diff , theClose , x , x1 , tIndex  )
                    for tIndex , (theXX1DiffEMA5 , theXX1Diff, theClose)
                    in enumerate( 
                        zip( df[f'{x}-{x1} EMA diff EMA 5'] , 
                             df[f'{x}-{x1} EMA diff'] ,
                             df["close"]   )   )   ]

            print(f'{x}-{x1} Diff EMA5 crossover Profit % {tickers[0]}')
#             print(f"Date : {finaltheDates[-1]}")
#             print(f"Balance : {finaltheBalance[-1]}")
#             print(f"Invested : {finaltheInvested[-1]}")
#             print(f"Multiplier : {finaltheShareMultiplier[-1]}")
            print(f"Profit : {df[f'{x}-{x1} Diff EMA5 crossover Profit %'][df.index[-1]]}")
            print("")
            print("")
                
    
print(stock_change())
print()


macd5Profit = MACD5Profit(100000.00 , 0, 0)

macd5Profit.macd5ProfitDFmake()



In [ ]:
# Bollinger Bands for upper and lower value of a ticker price using Simple Moving Average and Standard Deviation. 

def theSMA(df, x):
    df[f'SMA {x}'] = df["close"].rolling(window = x).mean()
    return df

def sd_sma(df, x):
    theSMA(df, x)
    df[f'SD of SMA {x}'] = df[f"SMA {x}"].rolling(window = x).std()
    return df

def bbx_upper_lower(df, x, y):
    sd_sma(df, x)
    df[f'BB{x} Upper'] = df[f"SMA {x}"] + y * df[f'SD of SMA {x}']   
    df[f'BB{x} Lower'] = df[f"SMA {x}"] - y * df[f'SD of SMA {x}']    
    return df

def dd_buy_sell_Threshold(df, theSMAs, theSMA_sd):
    for x,y in zip(theSMAs, theSMA_sd):
        bbx_upper_lower(df, x, y)

        buyThreshold = (df[f'SMA {x}']+df[f'BB{x} Lower'])/2
        buyThreshold = (buyThreshold+df[f'BB{x} Lower'])/2
        #better for sma 20 to do it third time otherwise better for sma 10
        df[f"buyThreshold{x}"] = buyThreshold

        sellThreshold = (df[f'SMA {x}']+df[f'BB{x} Upper'])/2
        sellThreshold = (sellThreshold+df[f'BB{x} Upper'])/2
        df[f"sellThreshold{x}"] = sellThreshold

    return df



theSMAs = [20,10,50]
theSMA_sd = [2.0,1.9,2.1]

dd_buy_sell_Threshold(df, theSMAs, theSMA_sd)

print()

In [ ]:
# Backtest: BB Profit/Loss by buying and selling when ticker close crosses upper and lower Bands
# Conclusion: Doesn't take into account how long ticker crosses over for. 

finaltheBalance = []
finaltheInvested = []
finaltheShareMultiplier = []
finaltheDates = []

class BBProfit:
    def __init__(self, aVal , bVal, cVal ):  
        self.tradeAccountBalance = []
        if len(self.tradeAccountBalance) == 0 : 
            self.tradeAccountBalance.append(aVal)
        self.aVal = aVal
        
        self.tradeAccountBalanceInitial = self.tradeAccountBalance[0]
        
        self.tradeAccountInvested = [] #theInvested
        if len(self.tradeAccountInvested) == 0 : 
            self.tradeAccountInvested.append(bVal)
        self.bVal = bVal
        
        self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

        self.shareCountMultiplier = []
        if len(self.shareCountMultiplier) == 0 : 
            self.shareCountMultiplier.append(cVal)
        self.cVal = cVal
        
        self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
        
        self.theDates = []
        self.theDates.append(f"{theStart} START")

    def calcProfit(self, ) :
        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        oBal = self.tradeAccountBalanceInitial
        oInv = self.tradeAccountInvestedInitial
    
        value = ( (((inv+bal) - (oBal+oInv)) / (oBal+oInv) ) ) * 100
        return value 
        
    def bbProfit(self, the_X_SMA , the_X_upper, the_X_lower, theClose , x , tIndex  ) :
        global finaltheBalance , finaltheInvested, finaltheShareMultiplier, finaltheDates

        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        mult = self.shareCountMultiplier[-1]
        
        #UPDATE INVESTMENT to current market value
        if mult >= 1 and inv > 0 : self.tradeAccountInvested[-1] = theClose*mult
        inv = self.tradeAccountInvested[-1]
        
        buyThreshold = (the_X_SMA + the_X_lower) / 2   
        buyThreshold = (buyThreshold + the_X_lower) / 2
        # Better for sma 20 to increase threshold, otherwise better for sma 10
#         buyThreshold = (buyThreshold + the_X_lower) / 2

        sellThreshold = (the_X_SMA + the_X_upper) / 2   
        sellThreshold = (sellThreshold + the_X_upper) / 2
        

        #BUY if the close is near bollinger lower band
        if ( (theClose < df[f"buyThreshold{x}"][df.index[tIndex]] ) 
            and (df['close'][df.index[tIndex-1]] > df[f"buyThreshold{x}"][df.index[tIndex-1]]) 
            and ( bal >= theClose) 
            and (df[f"buyThreshold{x}"][df.index[tIndex-1]] != numpy.nan) and (theClose != numpy.nan) ) :

            #Better buy price
#             if tIndex > 0 : theClose = df["close"][df.index[tIndex-1]]

            mult = int( bal / theClose )     
            bal = bal - theClose*mult - (theClose*mult*0.003)     #buy as many shares as balance available and fee
            inv = inv + theClose*mult

            if self.shareCountMultiplier[-1] > 0 :
                mult = mult + self.shareCountMultiplier[-1]

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} BUY")


        #Sell if the close is near bollinger upper band
        elif ( ((theClose > df[f"sellThreshold{x}"][df.index[tIndex]]) and ( inv > 0 ) 
                    and (df['close'][df.index[tIndex-1]] < df[f"sellThreshold{x}"][df.index[tIndex-1]]) 
                    and (df[f"sellThreshold{x}"][df.index[tIndex-1]] != numpy.nan) and (theClose != numpy.nan))
              
#                  #Stoploss
#                or ( (inv > 0) 
#                     and ((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*0.97 >= (inv+bal)) 
#                     and (self.tradeAccountInvested[-2] != numpy.nan) )
#                  #Takeprofit
#                or ( (inv > 0) 
#                     and ((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*1.1 < (inv+bal)) 
#                     and (self.tradeAccountInvested[-2] != numpy.nan) )
             )  :
            mult = int( inv / theClose )     
            bal = bal + inv - (theClose*mult*0.003) #fee
            inv = 0.00  
            mult = 0

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} SELL")

        else: pass

        #PROFIT calculator
        value = self.calcProfit()

        #RESET for next loop for different crossover
        if tIndex >= len(df) - 1 : 
            #self.indexPrint( tIndex )
            finaltheBalance.append(self.tradeAccountBalance[0:-1])
            self.tradeAccountBalance.clear()
            self.tradeAccountBalance.append(self.aVal)

            self.theBalanceInitial = self.tradeAccountBalance[0]

            finaltheInvested.append(self.tradeAccountInvested[0:-1])
            self.tradeAccountInvested.clear()
            self.tradeAccountInvested.append(self.bVal)

            self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

            finaltheShareMultiplier.append(self.shareCountMultiplier[0:-1])
            self.shareCountMultiplier.clear()
            self.shareCountMultiplier.append(self.cVal)

            self.shareCountMultiplierInitial = self.shareCountMultiplier[0]

            finaltheDates.append(self.theDates[0:-1])
            self.theDates.clear()
            self.theDates.append(f"{theStart} START")

        return value 
    
    def indexPrint(self, tIndex ) :
        print(tIndex)

    def bbProfitDFmake(self, ): 
        global finaltheBalance , finaltheInvested
        theSMAs = [20,10,50]

        for x in theSMAs :
            df[f'BB{x} Profit %'] =  [ 
                    self.bbProfit( the_X_SMA , the_X_upper, the_X_lower, theClose , x , tIndex  )
                       for tIndex , (the_X_SMA , the_X_upper, the_X_lower, theClose)
                       in enumerate( 
                           zip( df[f'SMA {x}'] ,
                                df[f'BB{x} Upper'] ,
                                df[f'BB{x} Lower'] ,  
                                df["close"]   )   )   ]

            print(f'BB{x} Profit % {tickers[0]}')
#             print(f"Date : {finaltheDates[-1]}")
#             print(f"Balance : {finaltheBalance[-1]}")
#             print(f"Invested : {finaltheInvested[-1]}")
#             print(f"Multiplier : {finaltheShareMultiplier[-1]}")
            print(f"Profit : {df[f'BB{x} Profit %'][df.index[-1]]}")
            print("")
            print("")
                

     
print(stock_change())
print()


bbProfit = BBProfit(100000.00 , 0, 0)

bbProfit.bbProfitDFmake()



In [ ]:
# Represent ticker and the indicators.

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LinearAxis, NumeralTickFormatter, Toggle, CheckboxGroup, RadioGroup, CustomJS
from bokeh.layouts import layout
from bokeh.layouts import column, row
from bokeh.models import Legend, Div
import pandas as pd

def candlestickWithVolume( df , plot_width = 600 , plot_height = 300 , x_range = None ) :
    dfTemp = df
    dfTempRSI = df

    dfTemp = dfTemp.reset_index()
    dfTempRSI = dfTemp.reset_index()
    dfTempRSI = dfTempRSI.fillna(0.00)
    
    source = ColumnDataSource( dfTemp )
    inc = ColumnDataSource( dfTemp[ dfTemp["Interval Status"] == "up" ]   )
    same = ColumnDataSource( dfTemp[ dfTemp["Interval Status"] == "same" ] )
    dec = ColumnDataSource( dfTemp[ dfTemp["Interval Status"] == "down" ] )
    
    theWidth= (dfTemp.index[1] - dfTemp.index[0]) / 2  

    fig = figure( plot_width = plot_width , 
                  plot_height = plot_height , 
                  x_range = x_range, 
                  y_range = ( min(dfTemp["low"]) * 0.975 , max(dfTemp["high"]) * 1.025 ) ,
                  y_axis_location = "right" , 
                  sizing_mode = "stretch_width"   )
    
    ### Axis and Label
    
    fig.xaxis.major_label_overrides = {
        i: pd.to_datetime(date).strftime("%Y-%m-%d %H:%M:%S") 
        for i, date in enumerate(source.data["Datetime"])
    }
    
    fig.xaxis.bounds = ( 0 , dfTemp.index[-1] ) 
    
    fig.yaxis.axis_label = "Price"
    fig.outline_line_color = "black"
    
    fig.extra_y_ranges = { 
        "volume" : Range1d( start = 0, end = max(dfTemp["volume"]) * 1.1 ) ,
        "rsi" : Range1d( start = 0, end = max(dfTempRSI["RSI"]) * 1.1 ) ,
        "macd" : Range1d( start = min(min(dfTempRSI["12-26 EMA diff"]), min(dfTempRSI["5-10 EMA diff"])) * 1.1 , 
                  end = max(max(dfTempRSI["12-26 EMA diff"]), max(dfTempRSI["5-10 EMA diff"])) * 1.1 )   }
        
    fig.add_layout( LinearAxis( y_range_name = "volume" , axis_label = "Volume" ) , "left"   )
    
    fig.yaxis.formatter = NumeralTickFormatter( format = "0,0" )
    
        
    fig.add_layout( LinearAxis( y_range_name = "rsi" , axis_label = "RSI" ) , "left"   )
    
    fig.add_layout( LinearAxis( y_range_name = "macd" , axis_label = "MACD" ) , "left"   )
    
    ### Volume
        
    fig.yaxis[0].visible = False
    
    volumeUp = fig.vbar( "index" , theWidth, "volume" ,   #x   #width   #top
                source = inc , y_range_name = "volume" , line_color = "black" , fill_color = "white" , 
                fill_alpha = 1 , line_alpha = 0.75 ,   visible = False , )
    
    volumeSame = fig.vbar( "index" , theWidth, "volume" ,          
                  source = same , y_range_name = "volume" , line_color = "black" , fill_color = "white" , 
                  fill_alpha = 1 , line_alpha = 0.75 ,   visible = False , )    
    
    volumeDown = fig.vbar( "index" , theWidth, "volume" ,         
                  source = dec  , y_range_name = "volume" , line_color = "grey" , fill_color = "grey" , 
                  fill_alpha = 1 , line_alpha = 1 ,   visible = False , ) 
    
    avgVol10 = fig.line( dfTemp.index , dfTemp[("AvgOfVolume10Int")] , visible = False , 
                y_range_name = "volume" , line_alpha = 0.5 , line_width = 1 , 
                line_color = '#ff358f' , line_dash = 'solid'   )

    toggle_vol = Toggle(label = "Volume", active = False , width = 100)
    toggle_vol.js_link('active', fig.yaxis[0] , 'visible')
    toggle_vol.js_link('active', volumeUp , 'visible')
    toggle_vol.js_link('active', volumeSame , 'visible')
    toggle_vol.js_link('active', volumeDown , 'visible')
    toggle_vol.js_link('active', avgVol10 , 'visible')

    ### Candlestick
    
    candleSegment = fig.segment( "index" , "high" , "index" , "low" , 
                     source = source , color = "black"   )
    
    candleInc = fig.vbar( "index" , theWidth, "open" , "close" , #x   #width   #top   #bottom
                 source = inc , line_color = "black" , fill_color = "white")

    candleSame = fig.vbar( "index" , theWidth, "open" , "close" ,       
                  source = same , line_color = "black" , fill_color = "black")
    
    candleDec = fig.vbar( "index" , theWidth, "open" , "close" ,      
                 source = dec , line_color = "black" , fill_color = "black")
    
    toggle_candle = Toggle(label = "Candles", active = True , width = 100)
    toggle_candle.js_link('active', candleSegment , 'visible')
    toggle_candle.js_link('active', candleInc , 'visible')
    toggle_candle.js_link('active', candleSame , 'visible')
    toggle_candle.js_link('active', candleDec , 'visible')
    
    ### Support and Resistance
    c_length = int( round( len(dfTemp["close"]) / 5 ) )
    r_y = dfTemp["close"].nlargest( c_length ).mean()
    s_y = dfTemp["close"].nsmallest( c_length ).mean()
        
    resistance_line = fig.line( "index" , y = r_y ,  source=source, visible = False ,
                        line_alpha = 0.75 , line_width = 1.5 , line_color = 'black' , line_dash = 'dotted'   )
    
    support_line = fig.line( "index" , y = s_y ,  source=source, visible = False ,
                    line_alpha = 0.75 , line_width = 1.5 , line_color = 'black' , line_dash = 'dotted'   )
        
    toggle_S_R_line = Toggle(label = "S&R" , active = False , width = 100 )
    toggle_S_R_line.js_link('active', resistance_line , 'visible')
    toggle_S_R_line.js_link('active', support_line , 'visible')
    
    ### RSI
    
    fig.yaxis[1].visible = False
    
    rsi_line = fig.line( "index" , "RSI",  source=source, visible = False , y_range_name = "rsi" ,
                line_alpha = 0.75 , line_width = 1.5 , line_color = '#ff7b00' , line_dash = 'dotted'   )
        
    toggle_rsi_line = Toggle(label = "rsi" , active = False , width = 100 )
    toggle_rsi_line.js_link('active', fig.yaxis[1] , 'visible')
    toggle_rsi_line.js_link('active', rsi_line , 'visible')

    ### 12-26 MACD 9
    
    fig.yaxis[2].visible = False
    
    macd9_12_26_diff_line = fig.line( "index" , "12-26 EMA diff" , source=source, visible = False , 
                             y_range_name = "macd" ,line_alpha = 0.5 , line_width = 1.5 , 
                             line_color = '#4424D6' , line_dash = 'dashed'   )
    macd9_12_26_diff_ema9_line = fig.line( "index" , "12-26 EMA diff EMA 9" , source=source, visible = False , 
                                  y_range_name = "macd" , line_alpha = 1 , line_width = 1.5 , 
                                  line_color = '#4424D6' , line_dash = 'solid'   )
        
    toggle_macd9_12_26 = Toggle(label = "12-26 MACD" , active = False , width = 100 )
    toggle_macd9_12_26.js_link('active', fig.yaxis[2] , 'visible')
    toggle_macd9_12_26.js_link('active', macd9_12_26_diff_line , 'visible')
    toggle_macd9_12_26.js_link('active', macd9_12_26_diff_ema9_line , 'visible')
    
    ### 5-10 MACD 9
    
    fig.yaxis[2].visible = False
    
    macd9_5_10_diff_line = fig.line( "index" , "5-10 EMA diff" , source=source,  visible = False , 
                            y_range_name = "macd" , line_alpha = 0.5 , line_width = 1.5 , 
                            line_color = '#00598a' , line_dash = 'dashed'   )
    
    macd9_5_10_diff_ema9_line = fig.line( "index" , "5-10 EMA diff EMA 9" , source=source,  visible = False , 
                                 y_range_name = "macd" , line_alpha = 1 , line_width = 1.5 , 
                                 line_color = '#00598a' , line_dash = 'solid'   )
        
    toggle_macd9_5_10 = Toggle(label = "5-10 MACD" , active = False , width = 100 )
    toggle_macd9_5_10.js_link('active', fig.yaxis[2] , 'visible')
    toggle_macd9_5_10.js_link('active', macd9_5_10_diff_line , 'visible')
    toggle_macd9_5_10.js_link('active', macd9_5_10_diff_ema9_line , 'visible')
    
    ### 12-26 MACD 5
    
    fig.yaxis[2].visible = False
    
    macd5_12_26_diff_line = fig.line( "index" , "12-26 EMA diff" , source=source, visible = False , 
                             y_range_name = "macd" , line_alpha = 0.5 , line_width = 1.5 , 
                             line_color = '#4424D6' , line_dash = 'dashed'   )
    macd5_12_26_diff_ema9_line = fig.line( "index" , "12-26 EMA diff EMA 5" , source=source, visible = False , 
                                  y_range_name = "macd" , line_alpha = 1 , line_width = 1.5 , 
                                  line_color = '#4424D6' , line_dash = 'solid'   )
        
    toggle_macd5_12_26 = Toggle(label = "12-26 MACD5" , active = False , width = 100 )
    toggle_macd5_12_26.js_link('active', fig.yaxis[2] , 'visible')
    toggle_macd5_12_26.js_link('active', macd5_12_26_diff_line , 'visible')
    toggle_macd5_12_26.js_link('active', macd5_12_26_diff_ema9_line , 'visible')
    
    ### 5-10 MACD 5
    
    fig.yaxis[2].visible = False
    
    macd5_5_10_diff_line = fig.line( "index" , "5-10 EMA diff" , source=source,  visible = False , 
                            y_range_name = "macd" , line_alpha = 0.5 , line_width = 1.5 , 
                            line_color = '#00598a' , line_dash = 'dashed'   )
    macd5_5_10_diff_ema9_line = fig.line( "index" , "5-10 EMA diff EMA 5" , source=source,  visible = False , 
                                 y_range_name = "macd" , line_alpha = 1 , line_width = 1.5 , 
                                 line_color = '#00598a' , line_dash = 'solid'   )
        
    toggle_macd5_5_10 = Toggle(label = "5-10 MACD5" , active = False , width = 100 )
    toggle_macd5_5_10.js_link('active', fig.yaxis[2] , 'visible')
    toggle_macd5_5_10.js_link('active', macd5_5_10_diff_line , 'visible')
    toggle_macd5_5_10.js_link('active', macd5_5_10_diff_ema9_line , 'visible')
    
    ### Bollinger Band 10
    
    pSMA_10_line = fig.line( "index" , "SMA 10" , source=source,  visible = False ,
                line_alpha = 1 , line_width = 1.5 , line_color = '#944300' , line_dash = 'solid'   )
    
    pBB_10_upper_line = fig.line( "index" , "BB10 Upper" , source=source, visible = False ,
                line_alpha = 0.5 , line_width = 1.5 , line_color = '#944300' , line_dash = 'dashed'   )
    
    pBB_10_lower_line = fig.line( "index" , "BB10 Lower" , source=source, visible = False ,
                line_alpha = 0.5 , line_width = 1.5 , line_color = '#944300' , line_dash = 'dashed'   )
        
    toggle_pBB_10 = Toggle(label = "BB SMA10" , active = False , width = 100 )
    toggle_pBB_10.js_link('active', pSMA_10_line , 'visible')
    toggle_pBB_10.js_link('active', pBB_10_upper_line , 'visible') 
    toggle_pBB_10.js_link('active', pBB_10_lower_line , 'visible')      
    
    ### Bollinger Band 20
    
    pSMA_20_line = fig.line( "index" , "SMA 20" , source=source,  visible = False ,
                line_alpha = 1 , line_width = 1.5 , line_color = '#944300' , line_dash = 'solid'   )
    
    pBB_20_upper_line = fig.line( "index" , "BB20 Upper" , source=source, visible = False ,
                line_alpha = 0.5 , line_width = 1.5 , line_color = '#944300' , line_dash = 'dashed'   )
    
    pBB_20_lower_line = fig.line( "index" , "BB20 Lower" , source=source, visible = False ,
                line_alpha = 0.5 , line_width = 1.5 , line_color = '#944300' , line_dash = 'dashed'   )
        
    toggle_pBB_20 = Toggle(label = "BB SMA20" , active = False , width = 100 )
    toggle_pBB_20.js_link('active', pSMA_20_line , 'visible')
    toggle_pBB_20.js_link('active', pBB_20_upper_line , 'visible') 
    toggle_pBB_20.js_link('active', pBB_20_lower_line , 'visible')  
    
    ### 5-10 EMA
    
    pEMA_5_line = fig.line( "index" , "EMA 5" , source=source, visible = False ,
                   line_alpha = 0.75 , line_width = 1.5 , line_color = '#FC600A' , line_dash = 'dashed'   )
    
    pEMA_10_line = fig.line( "index" , "EMA 10" , source=source, visible = False ,  
                    line_alpha = 1 , line_width = 1.5 , line_color = '#FC600A' )
    
#     legend_5_10 = Legend(items=[("EMA 5", [pEMA_5_line]) , ("EMA 10", [pEMA_10_line])])
#     legend_pEMA_5_10 = fig.add_layout(legend_5_10, 'below')
    
    toggle_pEMA_5_10 = Toggle( label = "EMA 5-10" , active = False , width = 100 )
    toggle_pEMA_5_10.js_link( 'active' , pEMA_5_line , 'visible' )
    toggle_pEMA_5_10.js_link( 'active' , pEMA_10_line , 'visible' )
#     toggle_pEMA_5_10.js_link('active', legend_5_10 , 'visible')
    
    ### 10-20 EMA
    
    pEMA_10_line = fig.line( "index" , "EMA 10" , source=source,  visible = False ,
                    line_alpha = 0.5 , line_width = 1.5 , line_color = '#944300' , line_dash = 'dashed'   )
    
    pEMA_20_line = fig.line( "index" , "EMA 20" , source=source, visible = False ,
                    line_alpha = 1 , line_width = 1.5 , line_color = '#944300'   )
        
    toggle_pEMA_10_20 = Toggle(label = "EMA 10-20" , active = False , width = 100 )
    toggle_pEMA_10_20.js_link('active', pEMA_10_line , 'visible')
    toggle_pEMA_10_20.js_link('active', pEMA_20_line , 'visible')

    ### 12-26 EMA
    
    pEMA_12_line = fig.line( "index" , "EMA 12" , source=source, visible = False ,
                    line_alpha = 0.5 , line_width = 1.5 , line_color = '#800080' , line_dash = 'dashed'   )
    
    pEMA_26_line = fig.line( "index" , "EMA 26", source=source, visible = False ,  
                    line_alpha = 1 , line_width = 1.5 , line_color = '#800080'   )
    
    toggle_pEMA_12_26 = Toggle(label = "EMA 12-26" , active = False , width = 100 )
    toggle_pEMA_12_26.js_link('active', pEMA_12_line , 'visible')
    toggle_pEMA_12_26.js_link('active', pEMA_26_line , 'visible')
    

    ### 20-50 EMA
    
    pEMA_20_line = fig.line( "index" , "EMA 20" , source=source,  visible = False , 
                    line_alpha = 0.5 , line_width = 1.5 , line_color = '#004d33' , line_dash = 'dashed'   )
    
    pEMA_50_line = fig.line( "index" , "EMA 50" , source=source,  visible = False ,
                    line_alpha = 1 , line_width = 1.5 , line_color = '#004d33'   )
    
    toggle_pEMA_20_50 = Toggle(label = "EMA 20-50" , active = False , width = 100 )
    toggle_pEMA_20_50.js_link('active', pEMA_20_line , 'visible')
    toggle_pEMA_20_50.js_link('active', pEMA_50_line , 'visible')
    
    ###
    

    return fig , toggle_candle, toggle_vol, toggle_S_R_line, toggle_rsi_line, toggle_macd9_12_26, toggle_macd9_5_10, toggle_macd5_12_26, toggle_macd5_5_10, toggle_pBB_10, toggle_pBB_20, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50


(fig , toggle_candle, toggle_vol, toggle_S_R_line, toggle_rsi_line, toggle_macd9_12_26, toggle_macd9_5_10, toggle_macd5_12_26, toggle_macd5_5_10, toggle_pBB_10, toggle_pBB_20, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50) = candlestickWithVolume( df[theStart:] , 0 )
output_notebook()
show( row( column( 
    row([fig]) , 
    row(toggle_candle, toggle_vol, toggle_S_R_line, toggle_rsi_line, toggle_macd9_12_26, toggle_macd9_5_10, toggle_macd5_12_26, toggle_macd5_5_10, ) , 
    row( toggle_pBB_10, toggle_pBB_20, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50) , 
    sizing_mode = "stretch_width" ) ) )



In [ ]:
# Add a cursor event to the graph then add a range tool

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LinearAxis, NumeralTickFormatter
from bokeh.models import CustomJS, CrosshairTool
from bokeh.models.annotations import Label
from bokeh import events
import pandas as pd

def display_event(source, xlabel, vlabel, plabel, pv_ratio):
    return CustomJS(args=dict(source=source, xlabel=xlabel, vlabel=vlabel, plabel=plabel, pv_ratio=pv_ratio), code="""
        try {
            xlabel.visible = true
            var date = new Date(source.data['Datetime'][Number(cb_obj['x']).toFixed(0)])
            xlabel.text = date.toISOString().substr(0, 19)
            xlabel.x = cb_obj['x']
            xlabel.y = 0
        } catch(e) {
            xlabel.visible = false
        }
        vlabel.visible = true
        vlabel.x = 0
        /*
        var yyy = cb_obj['y']
        vlabel.y = yyy
        vlabel.text = Math.round(yyy * pv_ratio ).toLocaleString()
        */
        plabel.visible = true
        plabel.x = cb_obj['origin'].inner_width
        plabel.y = cb_obj['y']
        plabel.text = Math.round(cb_obj['y']).toLocaleString()
    """)

def leave_event(xlabel, vlabel, plabel):
    return CustomJS(args=dict(xlabel=xlabel, vlabel=vlabel, plabel=plabel), code="""
        xlabel.visible = false
        vlabel.visible = false
        plabel.visible = false
    """)


dfTemp = df
target = df[theStart : ]
(fig , toggle_candle, toggle_vol, toggle_S_R_line, toggle_rsi_line, toggle_macd9_12_26, toggle_macd9_5_10, toggle_macd5_12_26, toggle_macd5_5_10, toggle_pBB_10, toggle_pBB_20, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50) = candlestickWithVolume( target )

xlabel = Label(y_units='screen', x_offset=-40, y_offset=-20, render_mode='css',
               border_line_color='black', background_fill_color='white', visible=False)
vlabel = Label(x_units='screen', x_offset=-76, y_offset=-8, render_mode='css',
               border_line_color='black', background_fill_color='white', visible=False)
plabel = Label(x_units='screen', x_offset=4, y_offset=-8, render_mode='css',
               border_line_color='black', background_fill_color='white', visible=False)
fig.add_layout(xlabel)
fig.add_layout(vlabel)
fig.add_layout(plabel)
pv_ratio = max(dfTemp.volume) * 1.1 / ((max(dfTemp.high) * 1.025))
fig.add_tools(CrosshairTool())
fig.js_on_event(events.MouseMove,
                display_event(source=ColumnDataSource(dfTemp), xlabel=xlabel, vlabel=vlabel, plabel=plabel, pv_ratio=pv_ratio))
fig.js_on_event(events.MouseLeave, leave_event(xlabel=xlabel, vlabel=vlabel, plabel=plabel))
reset_output()
# output_notebook()
# show( row( column( 
#     row([fig]) , 
#     row(toggle_candle, toggle_vol, toggle_S_R_line, toggle_rsi_line, toggle_macd9_12_26, toggle_macd9_5_10, toggle_macd5_12_26, toggle_macd5_5_10 ) , 
#     row(toggle_pBB_10, toggle_pBB_20, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50) , 
#     sizing_mode = "stretch_width" ) ) )


# Add a range tool to graph

from bokeh.models import RangeTool
from bokeh.layouts import column
from bokeh.plotting import reset_output
from bokeh.plotting import output_file

df_reset = df.reset_index()
# (fig , toggle_candle, toggle_vol, toggle_rsi_line, toggle_macd_12_26, toggle_macd_5_10, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50) = candlestickWithVolume( target , 1 )

(fig , toggle_candle, toggle_vol, toggle_S_R_line, toggle_rsi_line, toggle_macd9_12_26, toggle_macd9_5_10, toggle_macd5_12_26, toggle_macd5_5_10, toggle_pBB_10, toggle_pBB_20, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50) = candlestickWithVolume(df , x_range=(df_reset.index[-100], df_reset.index[-1]))

dfTemp_rangeTool = df
dfTempRSI_rangeTool = df

dfTemp_rangeTool = dfTemp_rangeTool.reset_index()
dfTempRSI_rangeTool = dfTempRSI_rangeTool.reset_index()
dfTempRSI_rangeTool = dfTempRSI_rangeTool.fillna(0.00)

source = ColumnDataSource(dfTemp_rangeTool)
fig.title.text = f'{tickers[0]}'
fig.title.align = 'center'
fig.title.text_font_size = '24pt'

xlabel = Label(y_units='screen', x_offset=-40, y_offset=-20, render_mode='css',
               border_line_color='black', background_fill_color='white', visible=False)
vlabel = Label(x_units='screen', x_offset=-76, y_offset=-8, render_mode='css',
               border_line_color='black', background_fill_color='white', visible=False)
plabel = Label(x_units='screen', x_offset=4, y_offset=-8, render_mode='css',
               border_line_color='black', background_fill_color='white', visible=False)

fig.add_layout(xlabel)
fig.add_layout(vlabel)
fig.add_layout(plabel)

pv_ratio = max(dfTemp_rangeTool.volume) * 2 / max(dfTemp_rangeTool.high)

fig.add_tools(CrosshairTool())

fig.js_on_event(events.MouseMove,
                display_event(source=source, xlabel=xlabel, vlabel=vlabel, plabel=plabel, pv_ratio=pv_ratio))
fig.js_on_event(events.MouseLeave, leave_event(xlabel=xlabel, vlabel=vlabel, plabel=plabel))


range_tool = RangeTool(x_range=fig.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select = figure(plot_height=130, y_range=fig.y_range, y_axis_location='right',
                tools="", toolbar_location=None, background_fill_color="#efefef")
select.line('index', 'close', source=source) 
select.extra_y_ranges = {"volume": Range1d(start=0, end=max(dfTemp_rangeTool.volume) * 2)}
select.add_layout(LinearAxis(y_range_name="volume"), 'left')
select.vbar('index', 0.5, 'volume', source=source, line_color="black", fill_color="black", y_range_name="volume")
select.yaxis.formatter = NumeralTickFormatter(format="0,0")
select.xaxis.major_label_overrides = {  
    i: pd.to_datetime(date).strftime("%Y-%m-%d %H:%M:%S") for i, date in enumerate(source.data['Datetime'])
}
select.add_tools(range_tool)

reset_output()
output_notebook() 
# output_file('soliton.html') 
# show(column(fig, select, sizing_mode='stretch_width'))  
show( row( column( 
    row([fig]) , 
    row(toggle_candle, toggle_vol, toggle_S_R_line, toggle_rsi_line, toggle_macd9_12_26, toggle_macd9_5_10, toggle_macd5_12_26, toggle_macd5_5_10 ) , 
    row(toggle_pBB_10, toggle_pBB_20, toggle_pEMA_5_10, toggle_pEMA_10_20, toggle_pEMA_12_26, toggle_pEMA_20_50) , 
    select ,
    sizing_mode = "stretch_width" ) ) )

In [ ]:
# rise2 or fall2 when Candle Rises or Falls two intervals in a row. 
# A precursor to candle identification.

def fallOrRise( b, theVal, tIndex ) :
    if tIndex >= 4 :
        if b == "Avg": df_temp = df['Price AvgOfInt']
        else: df_temp = df['close']
            
        if ( (df_temp[df.index[tIndex]] > df_temp[df.index[tIndex-1]])
                and (df_temp[df.index[tIndex-1]] > df_temp[df.index[tIndex-2]])   
           ) : value = "rise2"
            
        elif ( (df_temp[df.index[tIndex]] < df_temp[df.index[tIndex-1]] )
                  and ( df_temp[df.index[tIndex-1]] < df_temp[df.index[tIndex-2]] )   
             ) : value = "fall2"
            
        else: value = ""
            
        return value
    
    else: value = ""
    return value


def call_fallOrRise(df, AorC):
    for b in AorC:
        df[f"FallorRise for2d {b}Int"] = [ fallOrRise( b, theVal, tIndex )
                                          for tIndex , (theVal)
                                          in enumerate( df['Price AvgOfInt'] )   ]
    return df



AorC = ["Avg", "Clo"]

call_fallOrRise(df, AorC)

print()

In [ ]:
# Use talib to identify candles. 
# Didn't have much success with this as a strategy, but feel free to add the candlestick pattern as a condition. 
# Will replace this library with my own implementation later. 

import talib.abstract as ta

def bear_candle(df):
    # Bear 
    # df['CDL3BLACKCROWS'] = ta.CDL3BLACKCROWS(df) # -100 Bear
    # df['CDLABANDONEDBABY'] = ta.CDLABANDONEDBABY(df) # 100 Bear
    # df['CDLADVANCEBLOCK'] = ta.CDLADVANCEBLOCK(df) # -100 Bear
    # df['CDLDARKCLOUDCOVER'] = ta.CDLDARKCLOUDCOVER(df) # -100 Bear
    # df['CDLEVENINGDOJISTAR'] = ta.CDLEVENINGDOJISTAR(df) # -100 Bear
    # df['CDLEVENINGSTAR'] = ta.CDLEVENINGSTAR(df) # -100 Bear
    # df['CDLGRAVESTONEDOJI'] = ta.CDLGRAVESTONEDOJI(df) # 100 Bear
    df['CDLHANGINGMAN'] = ta.CDLHANGINGMAN(df) # -100 Bear
    # df['CDLIDENTICAL3CROWS'] = ta.CDLIDENTICAL3CROWS(df) # Bear
    # df['CDLSHOOTINGSTAR'] = ta.CDLSHOOTINGSTAR(df) # -100 Bear
    # df['CDLUPSIDEGAP2CROWS'] = ta.CDLUPSIDEGAP2CROWS(df) # 100 Bear
    return df

def bear_bull_candle(df):
    # Bear / Bull
    # df['CDL3INSIDE'] = ta.CDL3INSIDE(df) # -100 100 
    # df['CDL3LINESTRIKE'] = ta.CDL3LINESTRIKE(df) # -100 100 
    # df['CDL3OUTSIDE'] = ta.CDL3OUTSIDE(df) # -100 100 
    # df['CDL3STARSINSOUTH'] = ta.CDL3OUTSIDE(df) # -100 100
    # df['CDLBELTHOLD'] = ta.CDLBELTHOLD(df) # -100 100 
    # df['CDLBREAKAWAY'] = ta.CDLBREAKAWAY(df) # -100 100 
    # df['CDLCLOSINGMARUBOZU'] = ta.CDLCLOSINGMARUBOZU(df) # -100 100 
    # df['CDLCOUNTERATTACK'] = ta.CDLCOUNTERATTACK(df) # -100 100 
    # df['CDLENGULFING'] = ta.CDLENGULFING(df) # -100 100
    # df['CDLGAPSIDESIDEWHITE'] = ta.CDLGAPSIDESIDEWHITE(df) # 100
    # df['CDLHARAMI'] = ta.CDLHARAMI(df) # -100 100
    # df['CDLHARAMICROSS'] = ta.CDLHARAMICROSS(df) # -100 100
    # df['CDLHIGHWAVE'] = ta.CDLHIGHWAVE(df) # 100 -100
    # df['CDLHIKKAKE'] = ta.CDLHIKKAKE(df) # -200 -100 100 200
    # df['CDLHIKKAKEMOD'] = ta.CDLHIKKAKEMOD(df) # -200 -100 100 200
    # df['CDLINNECK'] = ta.CDLINNECK(df) # -100 100
    # df['CDLKICKING'] = ta.CDLKICKING(df) # -100 100
    # df['CDLKICKINGBYLENGTH'] = ta.CDLKICKINGBYLENGTH(df) # -100 100
    # df['CDLMARUBOZU'] = ta.CDLMARUBOZU(df) # -100 100
    # df['CDLMATHOLD'] = ta.CDLMATHOLD(df) # -100 100 
    # df['CDLRISEFALL3METHODS'] = ta.CDLRISEFALL3METHODS(df) # -100 100
    # df['CDLSEPARATINGLINES'] = ta.CDLSEPARATINGLINES(df) # -100 100
    # df['CDLSTALLEDPATTERN'] = ta.CDLSTALLEDPATTERN(df) # -100 100
    # df['CDLSTICKSANDWICH'] = ta.CDLSTICKSANDWICH(df) # -100 100
    # df['CDLTHRUSTING'] = ta.CDLTHRUSTING(df) # -100 100
    # df['CDLTRISTAR'] = ta.CDLTRISTAR(df) # -100 100
    # df['CDLXSIDEGAP3METHODS'] = ta.CDLXSIDEGAP3METHODS(df) # -100 100
    return df

def bull_candle(df):
    # Bull
    # df['CDL3WHITESOLDIERS'] = ta.CDL3WHITESOLDIERS(df) # 100 Bull
    # df['CDLCONCEALBABYSWALL'] = ta.CDLCONCEALBABYSWALL(df) # 100 Bull
    # df['CDLDRAGONFLYDOJI'] = ta.CDLDRAGONFLYDOJI(df) # 100 Bull
    df['CDLHAMMER'] = ta.CDLHAMMER(df) # 100 Bull
    # df['CDLHOMINGPIGEON'] = ta.CDLHOMINGPIGEON(df) # 100 Bull
    # df['CDLINVERTEDHAMMER'] = ta.CDLINVERTEDHAMMER(df) # 100 Bull
    # df['CDLLADDERBOTTOM'] = ta.CDLLADDERBOTTOM(df) # 100 Bull
    # df['CDLMATCHINGLOW'] = ta.CDLMATCHINGLOW(df) # 100 Bull
    # df['CDLMORNINGDOJISTAR'] = ta.CDLMORNINGDOJISTAR(df) # 100 Bull
    # df['CDLMORNINGSTAR'] = ta.CDLMORNINGSTAR(df) # 100 Bull
    # df['CDLPIERCING'] = ta.CDLPIERCING(df) # 100 Bull
    # df['CDLUNIQUE3RIVER'] = ta.CDLUNIQUE3RIVER(df) # 100 Bull
    return df

def other_candle(df): 
    # Other
    # df['CDL2CROWS'] = ta.CDL2CROWS(df) # Bear
    # df['CDLLONGLINE'] = ta.CDLLONGLINE(df) # -100 100
    # df['CDLSHORTLINE'] = ta.CDLSHORTLINE(df) # -100 100
    # df['CDLSPINNINGTOP'] = ta.CDLSPINNINGTOP(df) # -100 100
    # df['CDLTAKURI'] = ta.CDLTAKURI(df) # 100
    # df['CDLTASUKIGAP'] = ta.CDLTASUKIGAP(df) # -100 100
    # df['CDLDOJI'] = ta.CDLDOJI(df) # 100
    # df['CDLDOJISTAR'] = ta.CDLDOJISTAR(df) # -100 100
    # df['CDLLONGLEGGEDDOJI'] = ta.CDLLONGLEGGEDDOJI(df) # 100
    # df['CDLONNECK'] = ta.CDLONNECK(df) # -100 100
    # df['CDLRICKSHAWMAN'] = ta.CDLRICKSHAWMAN(df) # 100
    return df


bear_candle(df)
bear_bull_candle(df)
bull_candle(df)
other_candle(df)

print()

In [ ]:
# Didn't have much success with this as a strategy, but feel free to add the candlestick pattern as a condition. 
# Will replace this library with my own implementation later. 

def borbCandle(theVal, tIndex):
    if( 
#         (df['CDLHARAMI'][df.index[tIndex]] == 100 )
#         (df['CDLPIERCING'][df.index[tIndex]] == 100 )
#          (df['CDLMORNINGSTAR'][df.index[tIndex]] == 100 )
         (df['CDLHAMMER'][df.index[tIndex]] == 100 ) 
      ): value = "bullCan"
        
    elif( 
#         (df['CDLHARAMI'][df.index[tIndex]] == -100 )
#          (df['CDLDARKCLOUDCOVER'][df.index[tIndex]] == -100 )
#          (df['CDLEVENINGSTAR'][df.index[tIndex]] == -100 )
         (df['CDLHANGINGMAN'][df.index[tIndex]] == -100 )
        ): value = "bearCan"
        
    else: value = ""
    return value


def call_borbCandle(df):         
    df["BorB Candle"] = [ borbCandle( theVal, tIndex )
                                          for tIndex , (theVal)
                                          in enumerate( df['close'] )   ]
    return df



call_borbCandle(df)

print()

In [ ]:
# Backtest: MACD9 with fall-fall-buy and rise rise sell Profit/Loss, no candlestick pattern condition. 
# Conclusion: A successfull strategy for most tickers. 
# eg: ETH-USD fell 20% but with this strategy we could have gone up 11% in same period of 50d. 

finaltheBalance = []
finaltheInvested = []
finaltheShareMultiplier = []
finaltheDates = []


class MACDProfit:
    def __init__(self, aVal , bVal, cVal ):  
        self.tradeAccountBalance = []
        if len(self.tradeAccountBalance) == 0 : 
            self.tradeAccountBalance.append(aVal)
        self.aVal = aVal
        
        self.tradeAccountBalanceInitial = self.tradeAccountBalance[0]
        
        self.tradeAccountInvested = [] #theInvested
        if len(self.tradeAccountInvested) == 0 : 
            self.tradeAccountInvested.append(bVal)
        self.bVal = bVal
        
        self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

        self.shareCountMultiplier = []
        if len(self.shareCountMultiplier) == 0 : 
            self.shareCountMultiplier.append(cVal)
        self.cVal = cVal
        
        self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
        
        self.theDates = []
        self.theDates.append(f"{theStart} START")

    def calcProfit(self, ) :
        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        oBal = self.tradeAccountBalanceInitial
        oInv = self.tradeAccountInvestedInitial
    
        value = ( (((inv+bal) - (oBal+oInv)) / (oBal+oInv) ) ) * 100
        return value 
        
    def theXX1Diff_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex != 0:
            theXX1Diff_temp = df[f'{x}-{x1} EMA diff'][df.index[tIndex-1]]
            theXX1DiffEMA9_temp = df[f'{x}-{x1} EMA diff EMA 9'][df.index[tIndex-1]]
            if (theXX1DiffEMA9_temp != numpy.nan) and (theXX1Diff_temp < theXX1DiffEMA9_temp) :
                return True
            else: return False 
        else: return False 
                
    def theXX1DiffEMA9_isLessYesterday(self, x , x1 , tIndex ) :
        if tIndex != 0:
            theXX1Diff_temp = df[f'{x}-{x1} EMA diff'][df.index[tIndex-1]]
            theXX1DiffEMA9_temp = df[f'{x}-{x1} EMA diff EMA 9'][df.index[tIndex-1]]
            if (theXX1DiffEMA9_temp != numpy.nan) and (theXX1Diff_temp > theXX1DiffEMA9_temp) :
                return True
            else: return False 
        else: return False
        
    def macdProfit(self, theXX1DiffEMA9 , theXX1Diff , theClose , x , x1 , tIndex  ) :
        global finaltheBalance , finaltheInvested, finaltheShareMultiplier 

        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        mult = self.shareCountMultiplier[-1]
        
        #UPDATE INVESTMENT to current market value
        if mult >= 1 and inv > 0 : self.tradeAccountInvested[-1] = theClose*mult
        inv = self.tradeAccountInvested[-1]

        theXX1Diff_lessYest = self.theXX1Diff_isLessYesterday(x , x1 , tIndex)
        theXX1DiffEMA9_lessYest = self.theXX1DiffEMA9_isLessYesterday(x , x1 , tIndex)

        #BUY if EMA x turns greater than x1 market price
        if ( (theXX1Diff > theXX1DiffEMA9) and theXX1Diff_lessYest
#             and (df["BorB Candle"][df.index[tIndex]] == "bullCan")
            and ((df["FallorRise for2d AvgInt"][df.index[tIndex-1]] == "fall2")
                     and (df["FallorRise for2d AvgInt"][df.index[tIndex-2]] == "fall2")
                     and (df["FallorRise for2d AvgInt"][df.index[tIndex]] == "")
                )
            and (bal >= theClose) and (theXX1DiffEMA9 != numpy.nan)
#             and (df["AvgOfVolume10Int"][df.index[tIndex]]<df["volume"][df.index[tIndex]])
           ) :

            #Better buy price when tIndex-1 that is previous close
#             if tIndex > 0 : theClose = df["close"][df.index[tIndex-1]]

            mult = int( bal / theClose )     
            bal = bal - theClose*mult - (theClose*mult*0.003)    #buy as many shares as balance available
            inv = inv + theClose*mult

            if self.shareCountMultiplier[-1] > 0 :
                mult = mult + self.shareCountMultiplier[-1]

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} BUY")


        #SELL if EMA x turns lesser than x1 market price
        elif ( 
            ( (theXX1Diff < theXX1DiffEMA9) and theXX1DiffEMA9_lessYest
#                and (df["BorB Candle"][df.index[tIndex]] == "bearCan")
               and ((df["FallorRise for2d AvgInt"][df.index[tIndex-1]] == "rise2")
                        and (df["FallorRise for2d AvgInt"][df.index[tIndex-2]] == "rise2")
                        and (df["FallorRise for2d AvgInt"][df.index[tIndex]] == "")
                   )
               and (inv > 0) and (theXX1DiffEMA9 != numpy.nan) )
              
                #Stoploss
#                 or ( (inv > 0) 
#                     and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*0.97) >= (inv+bal)) 
#                     and (self.tradeAccountInvested[-2] != numpy.nan) )
                #Takeprofit
                or ( (inv > 0) 
                    and (((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*1.10) < (inv+bal)) 
                    and (self.tradeAccountInvested[-2] != numpy.nan) )
              
             )  :
            mult = int( inv / theClose )     
            bal = bal + inv - (theClose*mult*0.003)
            inv = 0.00  
            mult = 0

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} SELL")
        else:
            pass

        #PROFIT calculator
        value = self.calcProfit()

        #RESET for next loop for different crossover
        if tIndex >= len(df) - 1 : 
            #self.indexPrint( tIndex )
            finaltheBalance.append(self.tradeAccountBalance[0:-1])
            self.tradeAccountBalance.clear()
            self.tradeAccountBalance.append(self.aVal)

            self.theBalanceInitial = self.tradeAccountBalance[0]

            finaltheInvested.append(self.tradeAccountInvested[0:-1])
            self.tradeAccountInvested.clear()
            self.tradeAccountInvested.append(self.bVal)

            self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

            finaltheShareMultiplier.append(self.shareCountMultiplier[0:-1])
            self.shareCountMultiplier.clear()
            self.shareCountMultiplier.append(self.cVal)

            self.shareCountMultiplierInitial = self.shareCountMultiplier[0]

            finaltheDates.append(self.theDates[0:-1])
            self.theDates.clear()
            self.theDates.append(f"{theStart} START")

        return value 
    
    def indexPrint(self, tIndex ) :
        print(tIndex)

    def macdProfitDFmake(self, ): 
        global finaltheBalance , finaltheInvested
        theEMAsP = [12,5,5,10,20,50]
        theEMAs2P = [26,10,35,20,50,100]

        for x , x1 in zip(theEMAsP,theEMAs2P) :
            df[f'{x}-{x1} Diff EMA9 crossover Profit %'] =  [ 
                    self.macdProfit( theXX1DiffEMA9 , theXX1Diff , theClose , x , x1 , tIndex  )
                       for tIndex , (theXX1DiffEMA9 , theXX1Diff, theClose)
                       in enumerate( 
                           zip( df[f'{x}-{x1} EMA diff EMA 9'] , 
                                df[f'{x}-{x1} EMA diff'] ,
                                df["close"]   )   )   ]

            print(f'{x}-{x1} Diff EMA9 crossover Profit % {tickers[0]}')
#             print(f"Date : {finaltheDates[-1]}")
#             print(f"Balance : {finaltheBalance[-1]}")
#             print(f"Invested : {finaltheInvested[-1]}")
#             print(f"Multiplier : {finaltheShareMultiplier[-1]}")
            print(f"Profit : {df[f'{x}-{x1} Diff EMA9 crossover Profit %'][df.index[-1]]}")
            print("")
            print("")
                
    
    
print(stock_change())
print()

        
macdProfit = MACDProfit(10000.00 , 0, 0)

macdProfit.macdProfitDFmake()



In [ ]:
# Backtest: BB with fall-fall-buy and rise-rise-sell Profit/Loss, buying and selling at crossover+tred.
# Conclusion: Has success with some tickers. 
# A stoploss is detrement to this strategy, and a good sell signal might be enough. 

finaltheBalance = []
finaltheInvested = []
finaltheShareMultiplier = []
finaltheDates = []

class BBProfit:
    def __init__(self, aVal , bVal, cVal ):  
        self.tradeAccountBalance = []
        if len(self.tradeAccountBalance) == 0 : 
            self.tradeAccountBalance.append(aVal)
        self.aVal = aVal
        
        self.tradeAccountBalanceInitial = self.tradeAccountBalance[0]
        
        self.tradeAccountInvested = [] #theInvested
        if len(self.tradeAccountInvested) == 0 : 
            self.tradeAccountInvested.append(bVal)
        self.bVal = bVal
        
        self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

        self.shareCountMultiplier = []
        if len(self.shareCountMultiplier) == 0 : 
            self.shareCountMultiplier.append(cVal)
        self.cVal = cVal
        
        self.shareCountMultiplierInitial = self.shareCountMultiplier[0]
        
        self.theDates = []
        self.theDates.append(f"{theStart} START")

    def calcProfit(self, ) :
        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        oBal = self.tradeAccountBalanceInitial
        oInv = self.tradeAccountInvestedInitial
    
        value = ( (((inv+bal) - (oBal+oInv)) / (oBal+oInv) ) ) * 100
        return value 
        
    def bbProfit(self, theClose , x , tIndex  ) :
        global finaltheBalance , finaltheInvested, finaltheShareMultiplier, finaltheDates

        bal = self.tradeAccountBalance[-1]
        inv = self.tradeAccountInvested[-1]
        mult = self.shareCountMultiplier[-1]
        
        #UPDATE INVESTMENT to current market value
        if mult >= 1 and inv > 0 : self.tradeAccountInvested[-1] = theClose*mult
        inv = self.tradeAccountInvested[-1]        

        #BUY if the close is near bollinger lower band
        if ( (theClose < df[f"buyThreshold{x}"][df.index[tIndex]] ) 
                 and ( bal >= theClose) and (theClose != numpy.nan) 
            
                 and ((df["FallorRise for2d AvgInt"][df.index[tIndex-1]] == "fall2")
                          and (df["FallorRise for2d AvgInt"][df.index[tIndex-2]] == "fall2")
                          and (df["FallorRise for2d AvgInt"][df.index[tIndex]] == ""))
           ) :

            mult = int( bal / theClose )     
            bal = bal - theClose*mult - (theClose*mult*0.003)     #buy as many shares as balance available and fee
            inv = inv + theClose*mult

            if self.shareCountMultiplier[-1] > 0 :
                mult = mult + self.shareCountMultiplier[-1]

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} BUY")


        #Sell if the close is near bollinger upper band
        elif ( ((theClose > df[f"sellThreshold{x}"][df.index[tIndex]]) 
                    and ( inv > 0 ) and (theClose != numpy.nan)
                
                    and ((df["FallorRise for2d AvgInt"][df.index[tIndex-1]] == "rise2")
                            and (df["FallorRise for2d AvgInt"][df.index[tIndex-2]] == "rise2")
                            and (df["FallorRise for2d AvgInt"][df.index[tIndex]] == ""))
               
               )
              
#                  #Stoploss
#                or ( (inv > 0) 
#                     and ((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*0.97 >= (inv+bal)) 
#                     and (self.tradeAccountInvested[-2] != numpy.nan) )
                 #Takeprofit
               or ( (inv > 0) 
                    and ((self.tradeAccountInvested[-2]+self.tradeAccountBalance[-2])*1.1 < (inv+bal)) 
                    and (self.tradeAccountInvested[-2] != numpy.nan) )
             )  :
            mult = int( inv / theClose )     
            bal = bal + inv - (theClose*mult*0.003) #fee
            inv = 0.00  
            mult = 0

            self.tradeAccountBalance.append(bal)
            self.tradeAccountInvested.append(inv)
            self.shareCountMultiplier.append(mult)

            tDate = df.index[tIndex].strftime( '%Y-%m-%d %H:%M:%S' )
            self.theDates.append(f"{tDate} SELL")

        else: pass

        #PROFIT calculator
        value = self.calcProfit()

        #RESET for next loop for different crossover
        if tIndex >= len(df) - 1 : 
            #self.indexPrint( tIndex )
            finaltheBalance.append(self.tradeAccountBalance[0:-1])
            self.tradeAccountBalance.clear()
            self.tradeAccountBalance.append(self.aVal)

            self.theBalanceInitial = self.tradeAccountBalance[0]

            finaltheInvested.append(self.tradeAccountInvested[0:-1])
            self.tradeAccountInvested.clear()
            self.tradeAccountInvested.append(self.bVal)

            self.tradeAccountInvestedInitial = self.tradeAccountInvested[0]

            finaltheShareMultiplier.append(self.shareCountMultiplier[0:-1])
            self.shareCountMultiplier.clear()
            self.shareCountMultiplier.append(self.cVal)

            self.shareCountMultiplierInitial = self.shareCountMultiplier[0]

            finaltheDates.append(self.theDates[0:-1])
            self.theDates.clear()
            self.theDates.append(f"{theStart} START")

        return value 
    
    def indexPrint(self, tIndex ) :
        print(tIndex)

    def bbProfitDFmake(self, ): 
        global finaltheBalance , finaltheInvested
        theSMAs = [20,10,50]

        for x in theSMAs :
            df[f'BB{x} Profit %'] =  [ 
                    self.bbProfit(theClose , x , tIndex  )
                       for tIndex , (theClose)
                       in enumerate(df["close"]   )   ]

            print(f'BB{x} Profit % {tickers[0]}')
            print(f"Date : {finaltheDates[-1]}")
#             print(f"Balance : {finaltheBalance[-1]}")
#             print(f"Invested : {finaltheInvested[-1]}")
#             print(f"Multiplier : {finaltheShareMultiplier[-1]}")
            print(f"Profit : {df[f'BB{x} Profit %'][df.index[-1]]}")
            print("")
            print("")
                

     
print(stock_change())
print()


bbProfit = BBProfit(100000.00 , 0, 0)

bbProfit.bbProfitDFmake()

